In [1]:
import requests
import json
import csv
from datetime import datetime
import joblib
import re
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize, FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
import string
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

[nltk_data] Downloading package wordnet to /home/awchen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/awchen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
filename = "secrets/recipes-en-201706/epicurious-recipes_m2.json"
with open(filename, "r") as f:
    datastore = json.load(f)
    f.close()
datastore_df = pd.DataFrame(datastore)

In [4]:
datastore_df.head()

,id,dek,hed,pubDate,author,type,url,photoData,tag,aggregateRating,ingredients,prepSteps,reviewsCount,willMakeAgainPct,dateCrawled
0,54a2b6b019925f464b373351,How does fried chicken achieve No. 1 status? B...,Pickle-Brined Fried Chicken,2014-08-19T04:00:00.000Z,[],recipe,/recipes/food/views/pickle-brined-fried-chicke...,"{'id': '54a2b64a6529d92b2c003409', 'filename':...","{'category': 'ingredient', 'name': 'Chicken', ...",3.11,"[1 tablespoons yellow mustard seeds, 1 tablesp...",[Toast mustard and coriander seeds in a dry me...,7,100,1498547035
1,54a408a019925f464b3733bc,Spinaci all'Ebraica,Spinach Jewish Style,2008-09-09T04:00:00.000Z,[{'name': 'Edda Servi Machlin'}],recipe,/recipes/food/views/spinach-jewish-style-350152,"{'id': '56746182accb4c9831e45e0a', 'filename':...","{'category': 'cuisine', 'name': 'Italian', 'ur...",3.22,"[3 pounds small-leaved bulk spinach, Salt, 1/2...",[Remove the stems and roots from the spinach. ...,5,80,1498547740
2,54a408a26529d92b2c003631,"This majestic, moist, and richly spiced honey ...",New Year’s Honey Cake,2008-09-10T04:00:00.000Z,[{'name': 'Marcy Goldman'}],recipe,/recipes/food/views/majestic-and-moist-new-yea...,"{'id': '55e85ba4cf90d6663f728014', 'filename':...","{'category': 'cuisine', 'name': 'Jewish', 'url...",3.62,"[3 1/2 cups all-purpose flour, 1 tablespoon ba...",[I like this cake best baked in a 9-inch angel...,105,88,1498547738
3,54a408a66529d92b2c003638,The idea for this sandwich came to me when my ...,The B.L.A.Bagel with Lox and Avocado,2008-09-08T04:00:00.000Z,[{'name': 'Faye Levy'}],recipe,/recipes/food/views/the-b-l-a-bagel-with-lox-a...,"{'id': '5674617e47d1a28026045e4f', 'filename':...","{'category': 'cuisine', 'name': 'Jewish', 'url...",4.00,"[1 small ripe avocado, preferably Hass (see No...","[A short time before serving, mash avocado and...",7,100,1498547740
4,54a408a719925f464b3733cc,"In 1930, Simon Agranat, the chief justice of t...",Shakshuka a la Doktor Shakshuka,2008-09-09T04:00:00.000Z,[{'name': 'Joan Nathan'}],recipe,/recipes/food/views/shakshuka-a-la-doktor-shak...,"{'id': '56746183b47c050a284a4e15', 'filename':...","{'category': 'cuisine', 'name': 'Jewish', 'url...",2.71,"[2 pounds fresh tomatoes, unpeeled and cut in ...","[1. Place the tomatoes, garlic, salt, paprika,...",7,83,1498547740


In [5]:
datastore_df.drop([
          "pubDate",
          "author",
          "type",
          "aggregateRating",
          "reviewsCount",
          "willMakeAgainPct",
          "dateCrawled",
          'prepSteps'
        ],
        axis=1,
        inplace=True,
        )

datastore_df.rename({'url': 'recipe_url'}, axis=1, inplace=True)

In [10]:
concat = pd.concat([datastore_df.drop(["tag"], axis=1), datastore_df["tag"].apply(pd.Series)], axis=1)
concat.drop(
  [
      0,
      "photosBadgeAltText",
      "photosBadgeFileName",
      "photosBadgeID",
      "photosBadgeRelatedUri",
      "url"
  ],
  axis=1,
  inplace=True,
)

cuisine_only = concat[concat["category"] == "cuisine"]
cuisine_only.dropna(axis=0, inplace=True)

/home/awchen/anaconda3/envs/MeaLeon/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [11]:
cuisine_only

,id,dek,hed,recipe_url,photoData,ingredients,category,name
1,54a408a019925f464b3733bc,Spinaci all'Ebraica,Spinach Jewish Style,/recipes/food/views/spinach-jewish-style-350152,"{'id': '56746182accb4c9831e45e0a', 'filename':...","[3 pounds small-leaved bulk spinach, Salt, 1/2...",cuisine,Italian
2,54a408a26529d92b2c003631,"This majestic, moist, and richly spiced honey ...",New Year’s Honey Cake,/recipes/food/views/majestic-and-moist-new-yea...,"{'id': '55e85ba4cf90d6663f728014', 'filename':...","[3 1/2 cups all-purpose flour, 1 tablespoon ba...",cuisine,Jewish
3,54a408a66529d92b2c003638,The idea for this sandwich came to me when my ...,The B.L.A.Bagel with Lox and Avocado,/recipes/food/views/the-b-l-a-bagel-with-lox-a...,"{'id': '5674617e47d1a28026045e4f', 'filename':...","[1 small ripe avocado, preferably Hass (see No...",cuisine,Jewish
4,54a408a719925f464b3733cc,"In 1930, Simon Agranat, the chief justice of t...",Shakshuka a la Doktor Shakshuka,/recipes/food/views/shakshuka-a-la-doktor-shak...,"{'id': '56746183b47c050a284a4e15', 'filename':...","[2 pounds fresh tomatoes, unpeeled and cut in ...",cuisine,Jewish
22,54a408c36529d92b2c0036a1,Pomegranate juice adds tartness to this famous...,Apple and Pomegranate Tart Tartin,/recipes/food/views/apple-and-pomegranate-tart...,"{'id': '560d7911f9a841923089d7fe', 'filename':...","[1 cup all purpose flour, 2 tablespoons sugar,...",cuisine,French
...,...,...,...,...,...,...,...,...
34399,587fabfecd2508314395ad8c,"A whole chicken, poached in a ginger and anise...",Soy Sauce Chicken,/recipes/food/views/soy-sauce-chicken,"{'id': '587fa8d8b3246f1f0afae6a0', 'filename':...","[1 (3 1/4-lb/1.5-kg) whole chicken, 1 tablespo...",cuisine,Chinese
34417,588a417612082e6b03c03d27,"Sweet, spicy, and refreshing all in one, this ...",Cabbage Stir-Fry With Coconut and Lemon,/recipes/food/views/cabbage-stir-fry-with-coco...,"{'id': '5877b0363b8997bc108a4481', 'filename':...","[1/4 cup unsweetened grated coconut (fresh, fr...",cuisine,Indian
34432,58924da769cf1107794eb164,Serve this meatless main—made by frying torn-u...,Amaya’s Migas,/recipes/food/views/amayas-migas,"{'id': '589e007981a60af57a05cc4c', 'filename':...","[1 tablespoon vegetable oil, 2 cups dime-sized...",cuisine,Tex-Mex
34704,594006de68bd4e58aaab1d26,"Served with a side of plain jasmine rice, chic...",Home-Style Chicken Kebat,/recipes/food/views/home-style-burmese-chicken...,"{'id': '5942fbf4a76fcc25997a352f', 'filename':...","[1 1/2 pounds boneless, skinless chicken thigh...",cuisine,Southeast Asian


In [29]:
cuisine_only['name'].value_counts()

American                      5478
Italian                       2116
French                        1293
Asian                         1266
Mexican                        809
Indian                         355
Jewish                         353
Mediterranean                  323
Spanish/Portuguese             307
Middle Eastern                 289
Greek                          284
English                        225
Central American/Caribbean     189
Eastern European/Russian       167
Central/South American         150
Southwestern                   148
Cajun/Creole                   144
Scandinavian                   131
Thai                           120
Irish                          120
Moroccan                       117
Chinese                        115
African                         92
Japanese                        90
German                          79
Vietnamese                      57
Korean                           3
Tex-Mex                          3
Southern            

In [20]:
cuisines = set(cuisine_only['name'].values)

In [21]:
cuisines

{'African',
 'American',
 'Asian',
 'Cajun/Creole',
 'Central American/Caribbean',
 'Central/South American',
 'Chinese',
 'Eastern European/Russian',
 'English',
 'French',
 'German',
 'Greek',
 'Indian',
 'Irish',
 'Italian',
 'Japanese',
 'Jewish',
 'Korean',
 'Latin American',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Moroccan',
 'Scandinavian',
 'Southeast Asian',
 'Southern',
 'Southern Italian',
 'Southwestern',
 'Spanish/Portuguese',
 'Sushi',
 'Tex-Mex',
 'Thai',
 'Vietnamese'}

In [28]:
for cuis in cuisines:
    print(cuisine_only[cuisine_only['name'] == cuis].count())
    

id             1
dek            1
hed            1
recipe_url     1
photoData      1
ingredients    1
category       1
name           1
dtype: int64
id             189
dek            189
hed            189
recipe_url     189
photoData      189
ingredients    189
category       189
name           189
dtype: int64
id             120
dek            120
hed            120
recipe_url     120
photoData      120
ingredients    120
category       120
name           120
dtype: int64
id             120
dek            120
hed            120
recipe_url     120
photoData      120
ingredients    120
category       120
name           120
dtype: int64
id             289
dek            289
hed            289
recipe_url     289
photoData      289
ingredients    289
category       289
name           289
dtype: int64
id             131
dek            131
hed            131
recipe_url     131
photoData      131
ingredients    131
category       131
name           131
dtype: int64
id             115
dek    

In [27]:
cuisine_only['name'].count

<bound method Series.count of 1                Italian
2                 Jewish
3                 Jewish
4                 Jewish
22                French
              ...       
34399            Chinese
34417             Indian
34432            Tex-Mex
34704    Southeast Asian
34749           Japanese
Name: name, Length: 14832, dtype: object>